In [1]:
import os
#os.environ['PYSPARK_PYTHON'] = '/opt/conda/envs/py2/bin/python2 --jars /root/notebooks/tensorframes-assembly-0.2.4.jar,/root/notebooks/jpmml-sparkml-package-1.0-SNAPSHOT.jar'
#os.environ['SPARK_MASTER'] = 'spark://spark.datasticks.com:7077'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages databricks:tensorframes:0.2.3-s_2.11 pyspark-shell'

#os.environ['PYTHONPATH'] = '/root/notebooks/tensorframes-assembly-0.2.4.jar:/root/notebooks/jpmml-sparkml-package-1.0-SNAPSHOT.jar:/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor/python/:'
os.environ['PYTHONPATH'] = '/root/lib/jpmml-sparkml-package-1.0-SNAPSHOT.jar:$PYTHONPATH'

In [2]:
import sys
sys.path

['',
 '/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor/python',
 '/root/notebooks',
 '/opt/conda/envs/py2/lib/python27.zip',
 '/opt/conda/envs/py2/lib/python2.7',
 '/opt/conda/envs/py2/lib/python2.7/plat-linux2',
 '/opt/conda/envs/py2/lib/python2.7/lib-tk',
 '/opt/conda/envs/py2/lib/python2.7/lib-old',
 '/opt/conda/envs/py2/lib/python2.7/lib-dynload',
 '/opt/conda/envs/py2/lib/python2.7/site-packages/Sphinx-1.4.1-py2.7.egg',
 '/opt/conda/envs/py2/lib/python2.7/site-packages/setuptools-23.0.0-py2.7.egg',
 '/opt/conda/envs/py2/lib/python2.7/site-packages',
 '/opt/conda/envs/py2/lib/python2.7/site-packages/IPython/extensions',
 '/root/.ipython']

In [3]:
!export

export CLICOLOR='1'
export CONDA_DEFAULT_ENV='py3'
export GIT_PAGER='cat'
export HOME='/root'
export JPY_PARENT_PID='101'
export MPLBACKEND='module://ipykernel.pylab.backend_inline'
export PAGER='cat'
export PATH='/opt/conda/envs/py3/bin:/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin'
export PWD='/root/notebooks'
export PYTHONPATH='/root/lib/jpmml-sparkml-package-1.0-SNAPSHOT.jar:$PYTHONPATH'
export SHELL='/bin/bash'
export SPARK_HOME='/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor'
export TERM='xterm-color'
export USER='root'


In [4]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext

def getContexts(appName):
    if 'sparkContext' not in globals(): # You can't have more than one sqlContext running at a time
        global sparkContext
        global sqlContext

        import findspark
        findspark.init()

        sparkContext = SparkContext(appName=appName)
            #, pyFiles=['/root/notebooks/jpmml.py'])
        sqlContext = SQLContext(sparkContext)
    return (sparkContext, sqlContext)

In [5]:
(sparkContext, sqlContext) = getContexts('blah')

sparkContext

In [6]:
import sys
sys.path

['/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor/python',
 u'/tmp/spark-e3624844-7c5a-468c-a15a-ed47876a0d9f/userFiles-7b36a0e5-1b81-41ea-937f-0dc7775761d2',
 '/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor/python/lib/py4j-0.10.3-src.zip',
 '',
 '/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor/python',
 '/root/notebooks',
 '/opt/conda/envs/py2/lib/python27.zip',
 '/opt/conda/envs/py2/lib/python2.7',
 '/opt/conda/envs/py2/lib/python2.7/plat-linux2',
 '/opt/conda/envs/py2/lib/python2.7/lib-tk',
 '/opt/conda/envs/py2/lib/python2.7/lib-old',
 '/opt/conda/envs/py2/lib/python2.7/lib-dynload',
 '/opt/conda/envs/py2/lib/python2.7/site-packages/Sphinx-1.4.1-py2.7.egg',
 '/opt/conda/envs/py2/lib/python2.7/site-packages/setuptools-23.0.0-py2.7.egg',
 '/opt/conda/envs/py2/lib/python2.7/site-packages',
 '/opt/conda/envs/py2/lib/python2.7/site-packages/IPython/extensions',
 '/root/.ipython']

In [7]:
!export

export CLICOLOR='1'
export CONDA_DEFAULT_ENV='py3'
export GIT_PAGER='cat'
export HOME='/root'
export JPY_PARENT_PID='101'
export MPLBACKEND='module://ipykernel.pylab.backend_inline'
export PAGER='cat'
export PATH='/opt/conda/envs/py3/bin:/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin'
export PWD='/root/notebooks'
export PYSPARK_PYTHON='/opt/conda/envs/py2/bin/python'
export PYTHONPATH='/root/lib/jpmml-sparkml-package-1.0-SNAPSHOT.jar:$PYTHONPATH'
export SHELL='/bin/bash'
export SPARK_HOME='/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor'
export TERM='xterm-color'
export USER='root'


In [8]:
#import pyspark
#conf = pyspark.SparkConf()
#conf.setAppName('Yo!')
#sc = pyspark.SparkContext(master='local[*]', conf=conf, pyFiles=['/root/core.py', '/root/jpmml.py'])
#sqlContext = pyspark.SQLContext(sc)
#sc
#sqlContext

In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula
from pyspark.ml.regression import DecisionTreeRegressor

data = sqlContext.read.csv('file:///root/notebooks/data/wine.csv', header = True, inferSchema = True)

formula = RFormula(formula = "quality ~ .")
regressor = DecisionTreeRegressor()
pipeline = Pipeline(stages = [formula, regressor])
pipelineModel = pipeline.fit(data)

pipelineModel

PipelineModel_4a8086069364a16ed217

In [10]:
from jpmml import toPMMLBytes

pmmlBytes = toPMMLBytes(sparkContext, data, pipelineModel)

pmmlBytes

ImportError: No module named jpmml

In [ ]:
!ls /root

In [11]:
import tensorframes as tfs

ImportError: No module named tensorframes